In [1]:
import FINE as fn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%load_ext autoreload
%autoreload 2

# Water supply of a small mountain village

Two new houses (house 5 and 6) were built in a small mountain village in Utopia which requires an update of the existing clean water supply system of the village which now consists of 6 houses:

<img src="MountainVillage.png" style="width: 500px;"/>


### Water demand
The demand for clean water occurs in spring between 5 am and 11 pm, in summer between 4 am and 12 pm, in autumn between 5 am and 11 pm and in winter between 6 am and 11 pm. The demand for one house assumes random values between 0 to 1 Uh (Unit*hour) during the demand hours. These values are uniformly distributed and are 0 outside the demand hours.

### Water supply 
The water supply comes from a small tributary of a glacier river, which provides more water in summer and less in winter: the profile is given for each hour of the year as

f(t) = 8 \* sin(π*t/8760) + g(t)    
    
where g(t) is a uniformly distributed random value between 0 and 4.

### Water storage
Clean water can be stored in a water tank (newly purchased). The invest per capacity is 100€/Uh, the economic lifetime is 20 years.

### Water treatment
The river water is converted to clean water in a water treatment plant (newly purchased). The invest per capacity is 7000€/U, the economic lifetime is 20 years. Further, it needs some electricity wherefore it has operational cost of 0.05 €/U.

### Water transmission
The clean water can be transported via water pipes, where some already exist between the houses 1-4, the water treatment plant and thewater tank, however new ones might need tobe built to connect the newly built houses or reinforce the transmission along the old pipes. The invest for new pipes per capacity is 100 €/(m\*U), the invest if a new pipe route is built is 500 €/(m\*U), the economic lifetime is 20 years.


In [2]:
locations = ['House 1', 'House 2', 'House 3', 'House 4', 'House 5', 'House 6',
             'Node 1', 'Node 2', 'Node 3', 'Node 4', 'Water treatment', 'Water tank']
commodityUnitDict = {'clean water': 'U', 'river water': 'U'}
commodities = {'clean water', 'river water'}
numberOfTimeSteps=8760
hoursPerTimeStep=1

In [3]:
esM = fn.EnergySystemModel(locations=set(locations), commodities=commodities, numberOfTimeSteps=8760,
                           commodityUnitsDict=commodityUnitDict,
                           hoursPerTimeStep=1, costUnit='1e3 Euro', lengthUnit='m')

# Source

In [4]:
riverFlow = pd.DataFrame(np.zeros((8760,12)), columns=locations)
np.random.seed(42)
riverFlow.loc[:,'Water treatment'] = np.random.uniform(0,4,(8760))+8*np.sin(np.pi*np.arange(8760)/8760)

In [5]:
esM.add(fn.Source(esM=esM, name='River', commodity='river water', hasCapacityVariable=False,
                  operationRateMax=riverFlow, opexPerOperation = 0.05))

# Conversion

In [6]:
eligibility = pd.Series([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], index=locations)
esM.add(fn.Conversion(esM=esM, name='Water treatment plant', physicalUnit='U',
                      commodityConversionFactors={'river water':-1, 'clean water':1}, 
                      hasCapacityVariable=True, locationalEligibility=eligibility,
                      investPerCapacity=7, opexPerCapacity=0.02*7, interestRate=0.08, economicLifetime=20))

# Storage

In [7]:
eligibility = pd.Series([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], index=locations)
esM.add(fn.Storage(esM=esM, name='Water tank', commodity='clean water', hasCapacityVariable=True, 
                   chargeRate=1/24, dischargeRate=1/24, locationalEligibility=eligibility,
                   investPerCapacity=0.10, opexPerCapacity=0.02*0.1, interestRate=0.08, economicLifetime=20))

# Transmission

### Distances between eligible regions

In [8]:
distances = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 38, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 40, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 38, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 40, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 38, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 40, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 38, 40, 0, 105, 0, 0, 0, 0],
                      [0, 0, 38, 40, 0, 0, 105, 0, 100, 0, 0, 0],
                      [38, 40, 0, 0, 0, 0, 0, 100, 0, 30, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 30, 0, 20, 50],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 50, 0, 0]])

distances = pd.DataFrame(distances, index=locations, columns=locations)
distances

,House 1,House 2,House 3,House 4,House 5,House 6,Node 1,Node 2,Node 3,Node 4,Water treatment,Water tank
House 1,0,0,0,0,0,0,0,0,38,0,0,0
House 2,0,0,0,0,0,0,0,0,40,0,0,0
House 3,0,0,0,0,0,0,0,38,0,0,0,0
House 4,0,0,0,0,0,0,0,40,0,0,0,0
House 5,0,0,0,0,0,0,38,0,0,0,0,0
House 6,0,0,0,0,0,0,40,0,0,0,0,0
Node 1,0,0,0,0,38,40,0,105,0,0,0,0
Node 2,0,0,38,40,0,0,105,0,100,0,0,0
Node 3,38,40,0,0,0,0,0,100,0,30,0,0
Node 4,0,0,0,0,0,0,0,0,30,0,20,50


## Old water pipes

In [9]:
capacityFix = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 1, 1, 0, 0, 0, 0, 2, 0, 0, 0],
                        [1, 1, 0, 0, 0, 0, 0, 2, 0, 4, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 4, 4],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0]])

capacityFix = pd.DataFrame(capacityFix, index=locations, columns=locations)
capacityFix

,House 1,House 2,House 3,House 4,House 5,House 6,Node 1,Node 2,Node 3,Node 4,Water treatment,Water tank
House 1,0,0,0,0,0,0,0,0,1,0,0,0
House 2,0,0,0,0,0,0,0,0,1,0,0,0
House 3,0,0,0,0,0,0,0,1,0,0,0,0
House 4,0,0,0,0,0,0,0,1,0,0,0,0
House 5,0,0,0,0,0,0,0,0,0,0,0,0
House 6,0,0,0,0,0,0,0,0,0,0,0,0
Node 1,0,0,0,0,0,0,0,0,0,0,0,0
Node 2,0,0,1,1,0,0,0,0,2,0,0,0
Node 3,1,1,0,0,0,0,0,2,0,4,0,0
Node 4,0,0,0,0,0,0,0,0,4,0,4,4


The old pipes have many leckages wherefore they lose 0.1%/m of the water they transport.

In [10]:
isBuiltFix = capacityFix.copy()
isBuiltFix[isBuiltFix>0]=1

esM.add(fn.Transmission(esM=esM, name='Old water pipes', commodity='clean water',
                        losses = 0.1e-2,
                        distances=distances, hasCapacityVariable=True, hasIsBuiltBinaryVariable=True, bigM=100,
                        capacityFix=capacityFix, isBuiltFix=isBuiltFix))

## New water pipes

In [11]:
incidence = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0],
                      [0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0],
                      [1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]])

eligibility = pd.DataFrame(incidence, index=locations, columns=locations)
eligibility

,House 1,House 2,House 3,House 4,House 5,House 6,Node 1,Node 2,Node 3,Node 4,Water treatment,Water tank
House 1,0,0,0,0,0,0,0,0,1,0,0,0
House 2,0,0,0,0,0,0,0,0,1,0,0,0
House 3,0,0,0,0,0,0,0,1,0,0,0,0
House 4,0,0,0,0,0,0,0,1,0,0,0,0
House 5,0,0,0,0,0,0,1,0,0,0,0,0
House 6,0,0,0,0,0,0,1,0,0,0,0,0
Node 1,0,0,0,0,1,1,0,1,0,0,0,0
Node 2,0,0,1,1,0,0,1,0,1,0,0,0
Node 3,1,1,0,0,0,0,0,1,0,1,0,0
Node 4,0,0,0,0,0,0,0,0,1,0,1,1


The new are pipes are better but still lose 0.05%/m of the water they transport.

In [12]:
esM.add(fn.Transmission(esM=esM, name='New water pipes', commodity='clean water',
                        losses = 0.05e-2,
                        distances=distances, hasCapacityVariable=True, hasIsBuiltBinaryVariable=True, bigM=100,
                        locationalEligibility=eligibility,
                        investPerCapacity=0.1, investIfBuilt=0.5, interestRate=0.08, economicLifetime=50))

# Sink

In [13]:
winterHours=np.append(range(8520,8760),range(1920))
springHours, summerHours, autumnHours = np.arange(1920,4128), np.arange(4128,6384), np.arange(6384,8520)

demand = pd.DataFrame(np.zeros((8760,12)), columns=list(locations))
np.random.seed(42)
demand[locations[0:6]] = np.random.uniform(0,1,(8760,6))

demand.loc[winterHours[(winterHours%24 < 5) | (winterHours%24 >= 23)]] = 0
demand.loc[springHours[(springHours%24 < 4)]] = 0
demand.loc[summerHours[(summerHours%24 < 5) | (summerHours%24 >= 23)]] = 0
demand.loc[autumnHours[(autumnHours%24 < 6) | (autumnHours%24 >= 23)]] = 0

In [14]:
demand.sum().sum()

19955.41528977587

In [15]:
esM.add(fn.Sink(esM=esM, name='Water demand', commodity='clean water', hasCapacityVariable=False,
                  operationRateFix=demand))

# Optimize the system

In [16]:
esM.cluster(numberOfTypicalPeriods=7)


Clustering time series data with 7 typical periods and 24 time steps per period...
		(1.3906 sec)



In [17]:
esM.optimize(timeSeriesAggregation=True, optimizationSpecs='LogToConsole=1 OptimalityTol=1e-6 crossover=1')

Time series aggregation specifications:
Number of typical periods:7, number of time steps per periods:24

Declaring sets, variables and constraints for SourceSinkModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.6250 sec)

Declaring sets, variables and constraints for ConversionModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.0156 sec)

Declaring sets, variables and constraints for StorageModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.0625 sec)

Declaring sets, variables and constraints for TransmissionModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.1875 sec)

Declaring shared potential constraint...
		(0.0000 sec)

Declaring commodity balances...
		(0.2969 sec)

Declaring objective function...
		(0.0625 sec)

Using license file C:\Users\t.gross\gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file C:\Users\T58C8~1.

# Selected results output

### Sources and Sinks

In [18]:
esM.getOptimizationSummary("SourceSinkModel", outputLevel=2)

House 1  House 2  House 3  House 4  \
Component    Property  Unit                                               
River        TAC       [1e3 Euro/a]        0        0        0        0   
             operation [U*h/a]           NaN      NaN      NaN      NaN   
             opexOp    [1e3 Euro/a]      NaN      NaN      NaN      NaN   
Water demand operation [U*h/a]       3329.49  3301.98  3316.75  3353.06   

                                     House 5  House 6 Node 1 Node 2 Node 3  \
Component    Property  Unit                                                  
River        TAC       [1e3 Euro/a]        0        0      0      0      0   
             operation [U*h/a]           NaN      NaN    NaN    NaN    NaN   
             opexOp    [1e3 Euro/a]      NaN      NaN    NaN    NaN    NaN   
Water demand operation [U*h/a]       3321.53  3332.61    NaN    NaN    NaN   

                                    Node 4 Water tank Water treatment  
Component    Property  Unit                                            
River        TAC       [1e3 Euro/a]      0          0         1104.88  
             operation [U*h/a]         NaN        NaN         22097.7  
             opexOp    [1e3 Euro/a]    NaN        NaN         1104.88  
Water demand operation [U*h/a]         NaN        NaN             NaN

### Storage

In [19]:
esM.getOptimizationSummary("StorageModel", outputLevel=2)

House 1 House 2 House 3 House 4  \
Component  Property           Unit                                           
Water tank TAC                [1e3 Euro/a]       0       0       0       0   
           capacity           [U*h]            NaN     NaN     NaN     NaN   
           capexCap           [1e3 Euro/a]     NaN     NaN     NaN     NaN   
           invest             [1e3 Euro]       NaN     NaN     NaN     NaN   
           operationCharge    [U*h/a]          NaN     NaN     NaN     NaN   
           operationDischarge [U*h/a]          NaN     NaN     NaN     NaN   
           opexCap            [1e3 Euro/a]     NaN     NaN     NaN     NaN   

                                           House 5 House 6 Node 1 Node 2  \
Component  Property           Unit                                         
Water tank TAC                [1e3 Euro/a]       0       0      0      0   
           capacity           [U*h]            NaN     NaN    NaN    NaN   
           capexCap           [1e3 Euro/a]     NaN     NaN    NaN    NaN   
           invest             [1e3 Euro]       NaN     NaN    NaN    NaN   
           operationCharge    [U*h/a]          NaN     NaN    NaN    NaN   
           operationDischarge [U*h/a]          NaN     NaN    NaN    NaN   
           opexCap            [1e3 Euro/a]     NaN     NaN    NaN    NaN   

                                           Node 3 Node 4 Water tank  \
Component  Property           Unit                                    
Water tank TAC                [1e3 Euro/a]      0      0   0.700925   
           capacity           [U*h]           NaN    NaN    57.5226   
           capexCap           [1e3 Euro/a]    NaN    NaN    0.58588   
           invest             [1e3 Euro]      NaN    NaN    5.75226   
           operationCharge    [U*h/a]         NaN    NaN    1071.92   
           operationDischarge [U*h/a]         NaN    NaN    1071.92   
           opexCap            [1e3 Euro/a]    NaN    NaN   0.115045   

                                           Water treatment  
Component  Property           Unit                          
Water tank TAC                [1e3 Euro/a]               0  
           capacity           [U*h]                    NaN  
           capexCap           [1e3 Euro/a]             NaN  
           invest             [1e3 Euro]               NaN  
           operationCharge    [U*h/a]                  NaN  
           operationDischarge [U*h/a]                  NaN  
           opexCap            [1e3 Euro/a]             NaN

### Conversion

In [20]:
esM.getOptimizationSummary("ConversionModel", outputLevel=2)

House 1 House 2 House 3 House 4  \
Component             Property  Unit                                           
Water treatment plant TAC       [1e3 Euro/a]       0       0       0       0   
                      capacity  [U]              NaN     NaN     NaN     NaN   
                      capexCap  [1e3 Euro/a]     NaN     NaN     NaN     NaN   
                      invest    [1e3 Euro]       NaN     NaN     NaN     NaN   
                      operation [U*h/a]          NaN     NaN     NaN     NaN   
                      opexCap   [1e3 Euro/a]     NaN     NaN     NaN     NaN   

                                             House 5 House 6 Node 1 Node 2  \
Component             Property  Unit                                         
Water treatment plant TAC       [1e3 Euro/a]       0       0      0      0   
                      capacity  [U]              NaN     NaN    NaN    NaN   
                      capexCap  [1e3 Euro/a]     NaN     NaN    NaN    NaN   
                      invest    [1e3 Euro]       NaN     NaN    NaN    NaN   
                      operation [U*h/a]          NaN     NaN    NaN    NaN   
                      opexCap   [1e3 Euro/a]     NaN     NaN    NaN    NaN   

                                             Node 3 Node 4 Water tank  \
Component             Property  Unit                                    
Water treatment plant TAC       [1e3 Euro/a]      0      0          0   
                      capacity  [U]             NaN    NaN        NaN   
                      capexCap  [1e3 Euro/a]    NaN    NaN        NaN   
                      invest    [1e3 Euro]      NaN    NaN        NaN   
                      operation [U*h/a]         NaN    NaN        NaN   
                      opexCap   [1e3 Euro/a]    NaN    NaN        NaN   

                                             Water treatment  
Component             Property  Unit                          
Water treatment plant TAC       [1e3 Euro/a]          4.1662  
                      capacity  [U]                  4.88437  
                      capexCap  [1e3 Euro/a]         3.48238  
                      invest    [1e3 Euro]           34.1906  
                      operation [U*h/a]              22097.7  
                      opexCap   [1e3 Euro/a]        0.683811

### Transmission

In [21]:
esM.getOptimizationSummary("TransmissionModel", outputLevel=2)

House 1  House 2  \
Component       Property  Unit         LocationIn                          
New water pipes TAC       [1e3 Euro/a] House 1                0        0   
                                       House 2                0        0   
                                       House 3                0        0   
                                       House 4                0        0   
                                       House 5                0        0   
...                                                         ...      ...   
Old water pipes operation [U*h/a]      Node 2               NaN      NaN   
                                       Node 3           7.50041  4.55491   
                                       Node 4               NaN      NaN   
                                       Water tank           NaN      NaN   
                                       Water treatment      NaN      NaN   

                                                       House 3  House 4  \
Component       Property  Unit         LocationIn                         
New water pipes TAC       [1e3 Euro/a] House 1               0        0   
                                       House 2               0        0   
                                       House 3               0        0   
                                       House 4               0        0   
                                       House 5               0        0   
...                                                        ...      ...   
Old water pipes operation [U*h/a]      Node 2           7.2389  8.99169   
                                       Node 3              NaN      NaN   
                                       Node 4              NaN      NaN   
                                       Water tank          NaN      NaN   
                                       Water treatment     NaN      NaN   

                                                       House 5 House 6  \
Component       Property  Unit         LocationIn                        
New water pipes TAC       [1e3 Euro/a] House 1               0       0   
                                       House 2               0       0   
                                       House 3               0       0   
                                       House 4               0       0   
                                       House 5               0       0   
...                                                        ...     ...   
Old water pipes operation [U*h/a]      Node 2              NaN     NaN   
                                       Node 3              NaN     NaN   
                                       Node 4              NaN     NaN   
                                       Water tank          NaN     NaN   
                                       Water treatment     NaN     NaN   

                                                          Node 1    Node 2  \
Component       Property  Unit         LocationIn                            
New water pipes TAC       [1e3 Euro/a] House 1                 0         0   
                                       House 2                 0         0   
                                       House 3                 0  0.776557   
                                       House 4                 0  0.817429   
                                       House 5          0.776557         0   
...                                                          ...       ...   
Old water pipes operation [U*h/a]      Node 2                NaN       NaN   
                                       Node 3                NaN   93.6361   
                                       Node 4                NaN       NaN   
                                       Water tank            NaN       NaN   
                                       Water treatment       NaN       NaN   

                                                          Node 3   Node 4  \
Component       Property  Unit      

In [22]:
esM.componentModelingDict["TransmissionModel"].operationVariablesOptimum.sum(axis=1)

New water pipes  House 1          Node 3                 0.000000
                 House 2          Node 3                 0.000000
                 House 3          Node 2                 0.000000
                 House 4          Node 2                 0.000000
                 House 5          Node 1                 0.000000
                 House 6          Node 1                 0.000000
                 Node 1           House 5             3385.864780
                                  House 6             3400.619777
                                  Node 2                 0.000000
                 Node 2           House 3             3373.887533
                                  House 4             3412.683133
                                  Node 1              7162.516682
                                  Node 3                 0.000000
                 Node 3           House 1             3386.620539
                                  House 2             3364.900732
          